## Run AI reviewer for application


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env

from llm import create_client
from review import extract_pdf_text, perform_review
import pandas as pd

# Create a real client
api_key = os.getenv("OPENAI_API_KEY")
client, model = create_client("gpt-4o-2024-05-13")

app_paths = ["001"]
path = "docs/application/"
output_path = "docs/review/"

results = []
# feedback_results= []
for application in app_paths:
    print(f"Review for {application} was started...")
    # Output format
    result = {}

    # Load application and related document
    file_path = path + application
    application_text = extract_pdf_text(file_path + "/application.pdf")
    contract_text = extract_pdf_text(file_path + "/contract.pdf")

    # Call the function
    review = perform_review(
        application_text=application_text,
        contract_text=contract_text,
        model=model,
        client=client,
    )

    # Convert the dictionary to a string representation
    result["review_result"] = review
    table = pd.DataFrame.from_dict(review['comments'])
    review_text = table.to_markdown(index=False)

    # # Call the function
    # feedback = provide_feedback(
    #     application_text=app_docs,
    #     review_text=review_text,
    #     model=model,
    #     client=client,
    # )
    # result["feedback_result"] = feedback

    output_file = f"{output_path}{application}.md"
    # Save as a properly formatted Markdown file
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"# Review Result for Application z: {application}\n\n")
        
        f.write("## Review Result\n\n")
        f.write(table.to_markdown())
        f.write("```\n\n")

        # f.write("## Feedback\n\n")
        # f.write(feedback + "\n")

    print(f"Review for {application} was finished!")

# Print the result
print("Review was completed")

Using OpenAI API with model gpt-4o-2024-05-13.
Review for 001 was started...


In [19]:
pd.DataFrame.from_dict(review['comments'])

,Decision,LC description,Contract description
0,Not Mentioned,Percentage Credit Amount Tolerance: 10%,Not mentioned
1,Not Mentioned,GAFTA-approved Quality and Weight Certificate:...,GAFTA-approved Quality and Weight Certificate ...
2,Not Mentioned,Documents must be presented to the Negotiating...,Not mentioned
3,Not Mentioned,All banking charges outside Japan are for the ...,Not mentioned
4,Not Mentioned,Confirmation Instructions: Confirm,Not mentioned
5,Not Mentioned,Confirmation charges to be borne by Applicant,Not mentioned
6,Not Mentioned,Reimbursement by SWIFT is Acceptable,Not mentioned
7,Not Mentioned,Advising Bank’s Name & Address: BANK OF AMERIC...,Not mentioned
8,Contradicts,"Buyer: XYZ Imports Ltd., 5678 Shibuya Street, ...","Buyer: XYZ Imports Ltd., a company registered ..."


In [17]:
review['comments']

[{'Decision': 'Not Mentioned',
  'LC description': 'Percentage Credit Amount Tolerance: 10%',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Not Mentioned',
  'LC description': 'Confirmation Instructions: Confirm',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Not Mentioned',
  'LC description': 'Confirmation charges to be borne by Applicant',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Contradicts',
  'LC description': 'Latest Shipment Date: 2025/02/28',
  'Contract description': 'Shipment Period: As per bill(s) of lading dated or to be dated within January 1st 2025 to February 28th 2025'},
 {'Decision': 'Not Mentioned',
  'LC description': 'Documents must be presented to the Negotiating Bank within 8 days after the date of shipment.',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Not Mentioned',
  'LC description': 'All banking charges outside Japan are for the account of Beneficiary.',
  'Contract description': 'Not mentioned'}]